In [1]:
import os
os.chdir('/home/angel/Artificial_Intelligence/')

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Churn_Modelling.csv')

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


First, we build our array with the independant variables, thus we take all the columns except for the last one

In [5]:
X = df.iloc[:,3:-1].values

Now, our array with dependant variable

In [6]:
Y = df.iloc[:,-1].values

It is time to start our preprocessing data, here we handle missing data as well as categorical variables encoding.


In [7]:
df.count().isnull()

RowNumber          False
CustomerId         False
Surname            False
CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
Exited             False
dtype: bool

However, in the case of the current dataset we have not missing data thus we go straight to the categorical variables part.
We are going to use sklearn library to accomplish the labeling. (LabelEncoder and OneHotEncoder).

In [8]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])   #geography
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])   #gender
onehotencoder=OneHotEncoder(categorical_features=[1])
X=onehotencoder.fit_transform(X).toarray()

/home/angel/anaconda3/envs/cnn/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/angel/anaconda3/envs/cnn/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


Now we can split our dataset into train and test set, in this case, 20% of the dataset will be actually part of the test set

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=0)

We  almost finish , however,we need to scale our features, because most of the algorithms work under Euclidean distances.

In [10]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.fit_transform(X_test)

At this point, our data is ready to train our model. Hence, let's build it!

To do so, we are going to strongly rely on a library called Keras

In [11]:
from keras.models import Sequential #to initialize our Neuronal Network
from keras.layers import Dense #to create the layers in our NN

Using TensorFlow backend.


Firstly, we must initialize our ann, in this case we are going to work with Sequential (we are going to be adding up the layers)

In [12]:
classifier=Sequential()


Then we add the first layer : 
Units is the dimension of the output space
kernel_initializer refers to the kernel weights matrix
activation is the activation function (pretty common to use relu)
input_dim refers to the size of the input

In [13]:
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu',input_dim=12)) 

We add a second hidden layer 

In [14]:
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))


Now , we add the output layer, sigmoid is used for binary output(we could use Threshold as well but sigmoid is more flexible)

In [15]:
#Adding the output layer
classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

Once we have our ann, it's time to compile it, for this, we are going to use Stochastic Gradient Descent 

In [16]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Fitting the ANN to the training set, the batch size defines the number of samples that will be propagated through the network while the epochs are the number of cycles to perform

In [17]:
classifier.fit(X_train,Y_train, batch_size=10,epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 278us/step - loss: 0.4839 - acc: 0.7956
Epoch 2/100
8000/8000 [==============================] - 1s 174us/step - loss: 0.4297 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 178us/step - loss: 0.4248 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 154us/step - loss: 0.4210 - acc: 0.8121
Epoch 5/100
8000/8000 [==============================] - 1s 139us/step - loss: 0.4175 - acc: 0.8245
Epoch 6/100
8000/8000 [==============================] - 2s 201us/step - loss: 0.4156 - acc: 0.8264
Epoch 7/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.4143 - acc: 0.8297
Epoch 8/100
8000/8000 [==============================] - 2s 220us/step - loss: 0.4119 - acc: 0.8304
Epoch 9/100
8000/8000 [==============================] - 1s 126us/step - loss: 0.4113 - acc: 0.8314
Epoch 10/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.4102 - acc: 0.8310

Now let's predict based on our test set of independant variables

In [18]:
Y_pred=classifier.predict(X_test)
Y_pred=(Y_pred>0.5)

With the confusion matrix we can measure the accuracy of our model

In [19]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)

In [20]:
cm

array([[1549,   46],
       [ 268,  137]])

As we can see, our ann is not the best given the fact that it mispredicted 277 values as "yes"

Now, let's predict the outcome for a new client, we use the prebuilt function predict from the classifier. The input must be an array and we have to scale it

In [21]:
new_prediction=classifier.predict(sc_X.transform(np.array([[0,0.0,1,600,1,40,3,6000000,2,1,1,500000000]]))) #0.0 to say to tab we use float as type of value
new_prediction=(new_prediction>0.5)

In [22]:
new_prediction

array([[False]])

We could easily say that our work here is done, howver there is plenty of stuff left to do. Let's try to tune our ANN

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import Imputer,LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.metrics import confusion_matrix

def build_classifier(optimizer):
    classifier =  Sequential()
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu',input_dim=12))
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu'))
    classifier.add(Dense(units=1,kernel_initializer='uniform',
                     activation='sigmoid'))
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
    
    return classifier ##classifier is a Keras object

classifier = KerasClassifier(build_fn=build_classifier)
#DICTIONARY : Hyper-parameters, the algortihm will test the values given in the dictionary (huge computation cost)
parameters = {
        'batch_size':[25,35],#numbers of samples to be tested
        'epochs':[100,500], #numbers of cycles
        'optimizer':['adam','rmsprop']
        }
grid_search=GridSearchCV(estimator = classifier, param_grid=parameters, scoring='accuracy',cv=10 ) ##function to find the best parameter
grid_search=grid_search.fit(X_train,Y_train);
best_parameters= grid_search.best_params_
best_accuracy = grid_search.best_score_
cross_val_score(estimator=classifier, X=X_train,Y=Y_train,cv=10,n_jobs=5)

Epoch 1/100
7200/7200 [==============================] - 1s 150us/step - loss: 0.5579 - acc: 0.7971
Epoch 2/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4385 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.4308 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.4271 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.4241 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.4215 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.4195 - acc: 0.8101
Epoch 8/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4179 - acc: 0.8203
Epoch 9/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4166 - acc: 0.8253
Epoch 10/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4156 - acc: 0.8261
Epoch 11

Epoch 83/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.4007 - acc: 0.8362
Epoch 84/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.4004 - acc: 0.8367
Epoch 85/100
7200/7200 [==============================] - 1s 69us/step - loss: 0.4007 - acc: 0.8358
Epoch 86/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.4008 - acc: 0.8365
Epoch 87/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.4001 - acc: 0.8365
Epoch 88/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.4004 - acc: 0.8364
Epoch 89/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.4005 - acc: 0.8360
Epoch 90/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.4002 - acc: 0.8364
Epoch 91/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.4001 - acc: 0.8371
Epoch 92/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4006 - acc: 0.8368


Epoch 65/100
7200/7200 [==============================] - 0s 60us/step - loss: 0.3939 - acc: 0.8354
Epoch 66/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.3934 - acc: 0.8337
Epoch 67/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.3935 - acc: 0.8358
Epoch 68/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3924 - acc: 0.8350
Epoch 69/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.3932 - acc: 0.8361
Epoch 70/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.3926 - acc: 0.8379
Epoch 71/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.3931 - acc: 0.8362
Epoch 72/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3928 - acc: 0.8351
Epoch 73/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3930 - acc: 0.8367
Epoch 74/100
7200/7200 [==============================] - 1s 85us/step - loss: 0.3925 - acc: 0.8358


Epoch 47/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4046 - acc: 0.8346
Epoch 48/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.4045 - acc: 0.8357
Epoch 49/100
7200/7200 [==============================] - 0s 57us/step - loss: 0.4043 - acc: 0.8339
Epoch 50/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.4042 - acc: 0.8362
Epoch 51/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4043 - acc: 0.8336
Epoch 52/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.4042 - acc: 0.8339
Epoch 53/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.4042 - acc: 0.8350
Epoch 54/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.4040 - acc: 0.8339
Epoch 55/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.4036 - acc: 0.8344
Epoch 56/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4044 - acc: 0.8343


Epoch 29/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3350 - acc: 0.8615
Epoch 30/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.3344 - acc: 0.8633
Epoch 31/100
7200/7200 [==============================] - 0s 58us/step - loss: 0.3344 - acc: 0.8637
Epoch 32/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3345 - acc: 0.8644
Epoch 33/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.3344 - acc: 0.8651
Epoch 34/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.3336 - acc: 0.8639
Epoch 35/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.3340 - acc: 0.8624
Epoch 36/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.3335 - acc: 0.8628
Epoch 37/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3331 - acc: 0.8649
Epoch 38/100
7200/7200 [==============================] - 1s 69us/step - loss: 0.3330 - acc: 0.8639


...

7200/7200 [==============================] - 1s 74us/step - loss: 0.3972 - acc: 0.8354
Epoch 94/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3971 - acc: 0.8353
Epoch 95/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.3972 - acc: 0.8346
Epoch 96/100
7200/7200 [==============================] - 0s 61us/step - loss: 0.3972 - acc: 0.8357
Epoch 97/100
7200/7200 [==============================] - 0s 61us/step - loss: 0.3973 - acc: 0.8349
Epoch 98/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.3972 - acc: 0.8349
Epoch 99/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3972 - acc: 0.8362
Epoch 100/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3972 - acc: 0.8349
Epoch 1/100
7200/7200 [==============================] - 1s 166us/step - loss: 0.5672 - acc: 0.7925
Epoch 2/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.4489 - acc: 0.7938
Epoch 3/100
7

7200/7200 [==============================] - 1s 80us/step - loss: 0.4031 - acc: 0.8333
Epoch 76/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.4035 - acc: 0.8325
Epoch 77/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.4032 - acc: 0.8318
Epoch 78/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.4035 - acc: 0.8324
Epoch 79/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.4038 - acc: 0.8333
Epoch 80/100
7200/7200 [==============================] - 1s 85us/step - loss: 0.4033 - acc: 0.8324
Epoch 81/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4034 - acc: 0.8321
Epoch 82/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4034 - acc: 0.8340
Epoch 83/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4032 - acc: 0.8326
Epoch 84/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.4034 - acc: 0.8343
Epoch 85/100

7200/7200 [==============================] - 1s 78us/step - loss: 0.3407 - acc: 0.8639
Epoch 58/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3410 - acc: 0.8622
Epoch 59/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3399 - acc: 0.8611
Epoch 60/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3398 - acc: 0.8617
Epoch 61/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3396 - acc: 0.8629
Epoch 62/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3393 - acc: 0.8624
Epoch 63/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.3394 - acc: 0.8606
Epoch 64/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3387 - acc: 0.8629
Epoch 65/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.3388 - acc: 0.8618
Epoch 66/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3384 - acc: 0.8618
Epoch 67/100


Epoch 39/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3961 - acc: 0.8365
Epoch 40/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3957 - acc: 0.8378
Epoch 41/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3957 - acc: 0.8368
Epoch 42/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3959 - acc: 0.8383
Epoch 43/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.3955 - acc: 0.8375
Epoch 44/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3954 - acc: 0.8381
Epoch 45/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.3950 - acc: 0.8371
Epoch 46/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.3952 - acc: 0.8374
Epoch 47/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3945 - acc: 0.8374
Epoch 48/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3950 - acc: 0.8389


KeyboardInterrupt: 

In [24]:
Y_train

array([0, 0, 0, ..., 0, 0, 1])